In [ ]:
import os
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-01",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )    


In [ ]:
import cv2
from PIL import Image
import numpy as np
from tqdm import tqdm
import base64
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
#The original image is in tif format, we need to transfer it to jpg for GPT-4V to understand
TIF_PATH = "Replace with your path"
SAVE_PATH = "The path to save jpg from tif"
im_folder = os.listdir(TIF_PATH)
for i in im_folder:
    temp_path = os.path.join(TIF_PATH,i)
    im_path = os.listdir(temp_path)
    count = 0
    for j in tqdm(im_path):
        if j == "Thumbs.db": continue
        count += 1
        name = j.split("_")[0] + "_" + str(count) + ".jpg"
        image = Image.open(os.path.join(temp_path,j))
        jpg_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        cv2.imwrite(SAVE_PATH + name, jpg_image)        

In [ ]:
categories={
    0: "chaetognath",
    1: "larval fish",
    2: "Hydromedusa",
    3: "lobate ctenophore",
    4: "Pleurobrachia",
    5: "shrimp",
    6: "Siphonophore",
    7: "stomatopod larva",
    8: "Unknown",
    9: "Thaliac",
    10:"ctenophore"
}
possible_cls = ""
for i in categories.keys():
    possible_cls += categories[i] + ","
print(possible_cls)

In [ ]:
Result = []
IMAGE_PATH = "Your own path"
im_file = os.listdir(IMAGE_PATH)
for i in tqdm(im_file):
    im_path = os.path.join(IMAGE_PATH,i)
    image = encode_image(im_path)
    try:
        response = client.chat.completions.create(
                    model ="GPT-4V",
                messages=[
                    {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "You are a Marine Biologist. I will send you an image that contains zooplanktons. The possible classes are:" + possible_cls + " Please give me the most probable name shown by the given image in the following format: class name. Please just give a short answer and do not provide explanations for your choice."},
                        {
                        "type": "image_url",
                        "image_url": {
                            "url":  f"data:image/jpeg;base64,{image}"
                        },
                        },
                    ],
                    }
                ],
                max_tokens=50
                )
        Result = Result + [[i.split("_")[0],response.choices[0].message.content]]
    except Exception:
        Result = Result + [[i.split("_")[0],"error"]]
        continue